In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
cd '/content/drive/MyDrive/Dataset/Songs'

/content/drive/MyDrive/Dataset/Songs


In [ ]:
Bolly_rap = pd.read_csv("/content/drive/MyDrive/Dataset/Songs/Bollywood_Rap_features.csv")
Bolly_rap['labels'] = pd.Series([0 for x in range(len(Bolly_rap.index))], index=Bolly_rap.index)
Bolly_rap.shape

Ghazal = pd.read_csv("/content/drive/MyDrive/Dataset/Songs/Ghazal_features.csv")
Ghazal['labels'] = pd.Series([1 for x in range(len(Ghazal.index))], index=Ghazal.index)
Ghazal.shape

Garhwali = pd.read_csv("/content/drive/MyDrive/Dataset/Songs/Garhwali_features.csv")
Garhwali['labels'] = pd.Series([2 for x in range(len(Garhwali.index))], index=Garhwali.index)
Garhwali.shape

bhajan = pd.read_csv("/content/drive/MyDrive/Dataset/Songs/bhajan_features.csv")
bhajan['labels'] = pd.Series([3 for x in range(len(bhajan.index))], index=bhajan.index)
bhajan.shape

bolly_romantic = pd.read_csv("/content/drive/MyDrive/Dataset/Songs/Bollywood_Romantic_features.csv")
bolly_romantic['labels'] = pd.Series([4 for x in range(len(bolly_romantic.index))], index=bolly_romantic.index)
bolly_romantic.shape

sufi = pd.read_csv("/content/drive/MyDrive/Dataset/Songs/sufi_features.csv")
sufi['labels'] = pd.Series([5 for x in range(len(sufi.index))], index=sufi.index)
sufi.shape

bhojpuri = pd.read_csv("/content/drive/MyDrive/Dataset/Songs/bhojpuri_features.csv")
bhojpuri['labels'] = pd.Series([6 for x in range(len(bhojpuri.index))], index=bhojpuri.index)

In [ ]:
frame = pd.concat([Bolly_rap, Ghazal, Garhwali, bhajan, bolly_romantic, sufi, bhojpuri], ignore_index=True)
frame.head()

,Unnamed: 0,Song_Name,onset_strength,chroma_stft,chroma_cqt,chroma_cens,melspectrogram,mfcc,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_rolloff,tonnetz,zero_crossing_rate,labels
0,0,Bollywood_Rap/Bohemia - Dada.opus,1.359504,0.504378,0.431088,0.238247,12.234897,0.155766,3336.742281,3691.313529,19.326488,7111.706785,-0.029229,0.063615,0
1,1,Bollywood_Rap/Bohemia - Beparwah.opus,1.367846,0.481887,0.527467,0.255920,5.386636,1.173165,3813.465951,3903.036168,20.049207,8436.634644,0.027319,0.080023,0
2,2,Bollywood_Rap/Bohemia - Desi Put Javaan.opus,1.522658,0.614667,0.415879,0.241032,22.373873,4.658426,2062.671542,3053.880091,18.504100,4707.484604,-0.016368,0.024910,0
3,3,Bollywood_Rap/Bohemia - Dil Acapella.opus,1.428467,0.593779,0.524480,0.263806,17.920794,4.753779,2267.229849,3129.322550,20.090888,5376.694052,0.017713,0.041050,0
4,4,Bollywood_Rap/Bohemia - Aja Ni Aja.opus,1.541603,0.659266,0.580653,0.260237,22.393740,6.887264,4515.528428,4360.305689,17.931442,9524.892741,0.005748,0.078542,0


In [ ]:
final = frame.drop("Unnamed: 0", axis=1)
final.to_csv("all_features.csv")

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing

In [ ]:
X = final.drop(["Song_Name"], axis=1)
y = final.labels

In [ ]:
data = pd.read_csv("all_features.csv")
X = data.drop(["labels", "Unnamed: 0", "Song_Name"], axis=1)
X = preprocessing.scale(X)
y = data["labels"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(boosting_type='gbdt', learning_rate=0.2, min_child_samples=20)
clf.fit(X_train, y_train)

LGBMClassifier(learning_rate=0.2)

In [ ]:
y_pred=clf.predict(X_test)
print(y_pred)
print(y_test)

[0 3 1 2 4 2 4 6 0 3 1 0 4 4 5 2 4 5 0 2 0 5 0 4 2 6 4 2 5 0 6 6 0 2 1 5 4
 1 1 3 2 3 0 6 4 4 0 1 6 3 2 6 4 1 0 1 6 0 4 5 6 6 0 4 2 6 1 6 6 3 1 4 4 0
 0 0 3 6 2 6 4 4 0 5 6 2 5 0 2 0 0 6 1 6 0 5 5 2 1 1 1 5 1 6 2 1 6 6 2 0 2
 5 4 4 6 2 0 1 3]
30     0
299    3
177    1
204    2
354    4
      ..
580    6
228    2
84     0
140    1
394    4
Name: labels, Length: 119, dtype: int64


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

LightGBM Model accuracy score: 0.6471


In [ ]:
import pickle
filename = 'Best_GBM.pkl'
pickle.dump(clf.fit, open(filename, 'wb'))